In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
data = pd.read_csv('data/data.csv')
# concatenate song name and song lyric
data['lyric'] = data['song name'] + " " + data['song lyric']
# convert genre to numeric
mapping = {'COUNTRY': 0,
            'HIPHOP': 1,
            'INDIE': 2,
            'JAZZ': 3,
            'POP': 4,
            'ROCK': 5}
data['genre'] = data['category'].map(mapping)

lyric = data['lyric']
genre = data['genre']

# change
# split data into train, val, test
x_train, x_test, y_train, y_test = train_test_split(lyric, genre, test_size=0.056, random_state=43)
# split train into train and val
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.06, random_state=43)

c:\Users\Admin\anaconda3\envs\pytorch\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
len(x_train)

7986

In [ ]:
x_train.reset_index(drop=True, inplace=True)

In [13]:
x_train[44]

"Luckenbach, Texas (Back to the Basics of Love) (feat. Willie Nelson) The only two things in life that make it worth livin'. Is guitars that tune good and firm feelin' women. I don't need my name in the marquee lights. I got my song and I got you with me tonight. Maybe it's time we got back to the basics of love. Let's go to Luckenbach, Texas. With Waylon and Willie and the boys. This successful life we're livin'. Got us feuding like the Hatfields and McCoys. Between Hank Williams' pain songs and. Newberry's train songs and Blue Eyes Cryin' in the Rain. Out in Luckenbach, Texas ain't nobody feelin' no pain. So baby, let's sell your diamond ring. Buy some boots and faded jeans and go away. This coat and tie is choking me. In your high society you cry all day. We've been so busy keepin' up with the Jones. Four car garage and we're still building on. Maybe it's time we got back to the basics of love. . Let's go to Luckenbach, Texas. With Waylon and Willie and the boys. This successful lif

In [4]:
type(genre)

numpy.ndarray

In [5]:
genre.shape

(9000,)

In [6]:
from utils import get_dataloader

train_loader, valid_loader, test_loader = get_dataloader('data/data.csv')


c:\Users\Admin\anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Admin\anaconda3\envs\pytorch\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [7]:
sample = next(iter(train_loader))